# Importing necessary Libraries and Files(Temperature, Solar and Humidity Data)

In [ ]:
import pandas as pd
import numpy as np
temp_data = pd.read_csv("Temperature_Data(Cleaned).csv")
sol_data = pd.read_csv("Solar_Data(Cleaned).csv")
hum_data = pd.read_csv("Humidity_Data(Cleaned).csv")

# Converting the dataframe files to numpy array

In [ ]:
temp_data = temp_data.values
sol_data = np.round(sol_data.values, decimals=1)  #converting to array and rounding the values to 1 decimal place
hum_data = hum_data.values

# Generating normal random values of Building area and height

In [ ]:
min_area = 50
max_area = 1000
mean = (min_area+max_area)/2
std_dev = (max_area-min_area)/6
Building_area = np.random.normal(mean, std_dev, size=10000)
Building_area = np.clip(Building_area, min_area,  max_area)   # Clipping the values between max aand min
Building_area = np.round(Building_area/10) * 10               # Rounding the values to integer values of 10
print(Building_area)
print(Building_area.shape)

[580. 530. 430. ... 390. 450. 500.]
(10000,)


In [ ]:
min_height = 3
max_height = 15
mean = (min_height+max_height)/2
std_dev = (max_height-min_height)/6
Building_height = np.random.normal(mean, std_dev, size=10000)
Building_height = np.clip(Building_height, min_height,  max_height)   # Clipping the values between max aand min
Building_height = np.round(Building_height/3) * 3
print(Building_height)
print(Building_height.shape)

[ 6. 12.  9. ...  9. 12.  9.]
(10000,)


# Creating Building type array(Based on Census Data)

In [ ]:
Building_types = ["Single house", "Townhouse", "Duplex", "Apartment" ]
probabilities = [0.7, 0.06, 0.08, 0.16] #Based on Census Data of 2021 of Australia
Building_types = np.random.choice(Building_types, size=10000, p=probabilities)
print(Building_types)
print(Building_types.shape)

['Single house' 'Single house' 'Single house' ... 'Single house'
 'Single house' 'Single house']
(10000,)


# Calculating Energy Consuption

In [ ]:
Building_type_df = pd.get_dummies(Building_types) # Converting categorical data into dummy variables
Building_type_df = Building_type_df.values # Converting into np array

# Define coefficients for each input variable
Building_area_coefficient = 2.0
Building_height_coefficient = 1.0
Temperature_coefficient = 1.7
Humidity_coefficient = 0.7
Solar_energy_coefficient = 0.3

# Reshaping Building area and height arrays
Building_area = Building_area.reshape(-1,1)
Building_height = Building_height.reshape(-1,1)

# Calculate energy consumption based on the linear relationship
energy_consumption = (Building_area * Building_area_coefficient +
                      Building_height * Building_height_coefficient +
                      temp_data * Temperature_coefficient +
                      sol_data * Solar_energy_coefficient +
                      hum_data * Humidity_coefficient +
                      Building_type_df[:, 0].reshape(-1,1) * 1 +   # Coefficient for Single house
                      Building_type_df[:, 1].reshape(-1,1) * 5 +   # Coefficient for Townhouse
                      Building_type_df[:, 2].reshape(-1,1) * 2 +   # Coefficient for Duplex
                      Building_type_df[:, 3].reshape(-1,1) * 4)    # Coefficient for Apartment

# Making the range of enrrgy_consuption between the desired range
l = 50                          # Lower bound of desired range
u = 300                         # Upper bound of desired range
m1 = energy_consumption.min()
m2 = energy_consumption.max()
energy_consumption = ((energy_consumption - m1)/(m2 - m1)) * (u - l) + l

# Making energy consumption as multiples of 10
energy_consumption = np.round((energy_consumption/10)) * 10

print(energy_consumption)
print(energy_consumption.shape)

[[190.]
 [180.]
 [150.]
 ...
 [140.]
 [160.]
 [170.]]
(10000, 1)


# Combining all data into one Data Frame

In [ ]:
Housing_data = pd.DataFrame({
    "Building Area(m2)" : Building_area.flatten(),
    "Building_height(m)" : Building_height.flatten(),
    "Building Type" : Building_types.flatten(),
    "Temperature(degree C)" : temp_data.flatten(),
    "Relative Humidity(%)": hum_data.flatten(),
    "Solar Radiation(W/m2)" : sol_data.flatten(),
    "Energy Consumption(KW/m2/yr)" : energy_consumption.flatten()
})

print(Housing_data.head())

   Building Area(m2)  Building_height(m) Building Type  Temperature(degree C)  \
0              580.0                 6.0  Single house                   27.0   
1              530.0                12.0  Single house                   27.0   
2              430.0                 9.0  Single house                   27.0   
3              820.0                 6.0        Duplex                   28.0   
4              520.0                 6.0  Single house                   33.0   

   Relative Humidity(%)  Solar Radiation(W/m2)  Energy Consumption(KW/m2/yr)  
0                   100                   12.1                         190.0  
1                    84                   22.8                         180.0  
2                    91                   16.3                         150.0  
3                   100                   11.8                         250.0  
4                   100                   17.0                         180.0  


# Exporting the data to csv File

In [ ]:
Housing_data.to_csv("Housing data(generated).csv", index=False)